### Import Necessary Libraries

In [1]:
import os
import cv2
import json
import torch
import numpy as np
from tqdm import tqdm
from skimage.measure import label as ski_label, regionprops
from tqdm import tqdm
from tools.trans.mask_to_coco import build_polygon
from tools.visual.visual_whole import visualize_coco_segmentation

from holitracer.seg.engine import seg_predict_api
from holitracer.vector.engine import vector_predict_api
from holitracer.seg.models.unpernet import UPerNet
from holitracer.vector.models.base import VLRAsModel

torch version: 2.5.1, cuda version: 11.8


/home/faye/miniconda3/envs/vector/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/faye/miniconda3/envs/vector/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


### Define the function to vectorize one image

In [2]:
def process_image(image_path, result_dir, seg_model, vector_model, downsample_factors):
    
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
        
    # segmentation
    result_path, mask = seg_predict_api(
                                    model=seg_model,
                                    image_path=image_path,
                                    result_dir=result_dir,
                                    downsample_factors=downsample_factors,
                                )
    if mask is None:
        return
    
    # trans2coco
    polys = []
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    label_img = ski_label(mask > 0)
    props = regionprops(label_img)
    image_height, image_width = mask.shape
    for prop in tqdm(props, desc="Processing properties", total=len(props)):
        prop_mask = np.zeros_like(mask)
        prop_mask[prop.coords[:, 0], prop.coords[:, 1]] = 1
        padded_binary_mask = np.pad(
            prop_mask, pad_width=1, mode="constant", constant_values=0
        )

        contours, hierarchy = cv2.findContours(
            padded_binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS
        )
        
        poly = build_polygon(contours, hierarchy, 0, image_height, image_width)
        if poly is None:
            continue
        polys.append(poly)
        
    # vectorization
    refined_annotations = vector_predict_api(
        model=vector_model,
        image_path=image_path,
        polys=polys,
        d=25,
    )
    
    return refined_annotations

### Load the segmentation and vectorization model (take whubuilding as an example)
You need download corresponding model files from the [model zoo](./README.md)

In [3]:
seg_model = UPerNet(
    backbone='swin_l',
    nclass=2,
    isContext=True,
    pretrain=False
)
seg_model_path = "./data/models/whubuilding/seg/best_model.pth"
seg_model.load_state_dict(torch.load(seg_model_path))
seg_model.cuda()
seg_model.eval()

vector_model = VLRAsModel(
    num_points=32,
    backbone_path=seg_model_path,
    vlr_num=4
)
vector_model_path = "./data/models/whubuilding/vector/best_model.pth"
vector_model.load_state_dict(torch.load(vector_model_path))
vector_model.cuda()
vector_model.eval()

VLRAsModel(
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=192, out_features=576, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=192, out_features=192, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=192, out_features=768

### Predict on one image and visualiza the result.
You need to prepare an image from the [dataset](./README.md) as input.

In [ ]:
image_path = "./data/datasets/WHU_building_dataset/test/img/150000_220000.jpg"
coco_result = process_image(image_path, "./data/results", seg_model, vector_model, [1, 3, 6])
json.dump(coco_result, open("./data/results/150000_220000.json", "w"), indent=4)
# visualize
visualize_coco_segmentation(
    image_path=image_path,
    json_path="./data/results/150000_220000.json",
    save_dir="./data/results/"
)

Skipping 150000_220000, already exists.


Processing vectorization: 100%|██████████| 476/476 [00:46<00:00, 10.25it/s]


已保存可视化结果至: ./data/results/150000_220000.jpg
